<a href="https://colab.research.google.com/github/Erenoa6621/web_novel_analysis/blob/master/novelupRead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os,time,datetime
from bs4 import BeautifulSoup
from urllib.request import urlopen,urljoin,urlparse

from google.colab import auth
from oauth2client.client import  GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

BOOKNAME = "ノベルUP小説集"
try:
  book = gc.open(BOOKNAME)
except:
  book = gc.create(BOOKNAME)

In [ ]:
BASEURL = 'https://novelup.plus/story/'
NCODE = '825445162'
url = f"{BASEURL}{NCODE}"

html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')
novel_title = soup.find("div",class_="novel_title").get_text(strip=True)
author = soup.find("div",class_="novel_author").get_text(strip=True)
s = len(soup.find_all("div",class_="episode_link episode_show_visited"))
print(novel_title)

try:
  sheet = book.worksheet(novel_title[:16])
except:
  sheet = book.add_worksheet(novel_title[:16],5+s,6) 

In [ ]:
sheet.update_cell(1,1,"タイトル")
sheet.update_cell(1,2,"作品名："+novel_title)
sheet.update_acell("A2","著者")
sheet.update_acell("B2","作者："+author)
sheet.update_acell("A3","NCODE")
sheet.update_acell("B3",NCODE)

In [ ]:
i = 4
count = 0
sheet.append_row(["更新","サブタイトル","前書き","あとがき","本文","URL"])
file = "Load.txt"
for data in soup.find_all("li",class_=""):
  if s == count:
    break
  subtitle = data.select_one("a").get_text(strip=True)
  update = data.select("span")[1].text
  texturl = urljoin(url,data.a['href'])
  print(subtitle,update,texturl)
  count+=1
  
  textdata = urlopen(texturl)
  textsoup = BeautifulSoup(textdata,"html.parser")
  text = textsoup.find("div",class_="content").get_text(strip=True)

  text = text.replace('\n', '')

  preface = ''
  for tp in textsoup.select("div#novel_p"):
    preface += tp.get_text(strip=True)
  afterword = ''
  for aw in textsoup.select("div#novel_a"):
    afterword += aw.get_text(strip=True)
  list =[update,subtitle,preface,afterword,text,texturl]
  i+=1
  for j,data in enumerate(list):
    sheet.update_cell(i,j+1,data) 
    
  fileobj = open(file, "w", encoding = "utf_8")
  fileobj.write(subtitle);
  fileobj.write(text);
